# Synthesize a coloring book

In [52]:
import numpy as np
import skimage
from PIL import Image
from PIL.Image import Image as PilImage

from diffusers import StableDiffusionPipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoImageProcessor, Swin2SRForImageSuperResolution
import torch

import openai

# !pip install python-docx 
from docx import Document
from docx.shared import Inches, Mm
import re

## Define keys and load models

In [4]:
# openAI key
openai.api_key = ""
# for azure openAI
# your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_base =  "" 
openai.api_version = '2022-12-01' # this may change in the future
openai.api_type = 'azure'
#This will correspond to the custom name you chose for your deployment when you deployed a model. 
deployment_name='' 

def load_prompter():
    prompter_model = AutoModelForCausalLM.from_pretrained("microsoft/Promptist")
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"
    return prompter_model, tokenizer

prompter_model, prompter_tokenizer = load_prompter()

def load_upscaler():
    processor = AutoImageProcessor.from_pretrained("caidas/swin2SR-classical-sr-x2-64")
    model = Swin2SRForImageSuperResolution.from_pretrained("caidas/swin2SR-classical-sr-x2-64")
    return model, processor
    
upscale_model, upscale_processor = load_upscaler()

model_id = "dreamlike-art/dreamlike-photoreal-2.0"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

## Inputs

In [64]:
title = "The Zoo Animals Playing Sports Coloring Book"
topic = "anthropormophic zoo animals playing sports"
number_pages = 50
images_to_generate = 5
# upscale image
enhance_images = False

## Helper functions

In [67]:
# create a sketch for coloring based in an input image
def generate_coloring_page(input: PilImage) -> PilImage:
    # Convert to grayscale if needed
    if input.mode != "L":
        input = input.convert("L")
    np_image = np.asarray(input)
    
    # detect edges
    np_image = skimage.filters.sobel(np_image)
    # convert to 8 bpp
    np_image = skimage.util.img_as_ubyte(np_image)
    # Invert to get dark edges on a light background
    np_image = 255 - np_image
    # Improve the contrast
    np_image = skimage.exposure.rescale_intensity(np_image)
    
    return Image.fromarray(np_image)

# Create ideas for each desired page of the coloring book
def generate_coloring_book_page_descriptions(topic, number_pages):
    prompt = "Create "+str(number_pages)+" ideas for a coloring book on the topic of "+topic+"/n/nIdeas for coloring book topic:"
    response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.7,
            max_tokens=1500
        )
    result = response.choices[0].text.strip()
    return result, prompt

# Create fun facts about the image
def generate_fun_facts(topic):
    prompt = "Create fun facts about "+topic+"/n/nFun facts:"
    response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.7,
            max_tokens=1500
        )
    result = remove_after_create(response.choices[0].text.strip())
    return result

# Create fun facts about the image
def generate_summary(prompt: str):
    response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.5,
            max_tokens=1500
        )
    result = response.choices[0].text.strip()
    return result

# parse a string to list based on return characters
def parse_string(input_string: str) -> list:
    result = []
    lines = input_string.split('\n')
    for line in lines:
        if not 'Create ' in line and not line == "":
            result.append(remove_number_period(line.strip()))
    return result

def remove_after_create(s: str) -> str:
    keyword = "Create "
    index = s.find(keyword)
    
    if index == -1:
        # If "Create " is not found, return the original string
        return s
    
    # Find the end index of the keyword
    end_index = index + len(keyword)
    
    # Return the string up to the end of the keyword
    return s[:end_index]

def remove_number_period(s: str) -> str:
    # Use a regular expression to match a number followed by a period and a space
    pattern = r'^\d+\.\s'
    
    # Remove the matched pattern and return the modified string
    return re.sub(pattern, '', s).strip()

def generate(plain_text: str) -> str:
    input_ids = prompter_tokenizer(plain_text.strip()+" Rephrase:", return_tensors="pt").input_ids
    eos_id = prompter_tokenizer.eos_token_id
    outputs = prompter_model.generate(input_ids, do_sample=False, max_new_tokens=75, num_beams=8, num_return_sequences=8, eos_token_id=eos_id, pad_token_id=eos_id, length_penalty=-1.0)
    output_texts = prompter_tokenizer.batch_decode(outputs, skip_special_tokens=True)
    res = output_texts[0].replace(plain_text+" Rephrase:", "").strip()
    return res

def enhance_image(image):
    # prepare image for the model
    inputs = upscale_processor(image, return_tensors="pt")

    # forward pass
    with torch.no_grad():
        outputs = upscale_model(**inputs)

    # postprocess
    output = outputs.reconstruction.data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.moveaxis(output, source=0, destination=-1)
    output = (output * 255.0).round().astype(np.uint8)  # float32 to uint8
    
    return Image.fromarray(output)

## Workflow

In [60]:
# Create page ideas given the desired topic
page_ideas = generate_coloring_book_page_descriptions(topic, number_pages)
print(page_ideas)

('Anthropormophic Zoo Animals Playing Sports \n\n1. A monkey playing football\n2. An elephant playing hockey\n3. A giraffe playing baseball\n4. A rhino playing cricket\n5. A bear playing volleyball\n6. A lion playing rugby\n7. A fox playing kickball\n8. A hippo playing badminton\n9. A koala playing basketball\n10. A tiger playing soccer\n11. A parrot playing lacrosse\n12. A hippo playing tennis\n13. A koala playing dodgeball\n14. A bear playing ultimate frisbee\n15. A lion playing squash\n16. A cheetah playing golf\n17. A gorilla playing softball\n18. A fox playing curling\n19. A kangaroo playing handball\n20. A crocodile playing water polo\n21. A monkey playing table tennis\n22. An elephant playing billiards\n23. A giraffe playing darts\n24. A rhino playing disc golf\n25. A bear playing bowling\n26. A lion playing badminton\n27. A fox playing cricket\n28. A hippo playing croquet\n29. A koala playing hockey\n30. A tiger playing rugby\n31. A parrot playing basketball\n32. A hippo playin

In [61]:
# parse the page ideas into a list for each page
pages = parse_string(page_ideas[0])
print(pages)

['Anthropormophic Zoo Animals Playing Sports', 'A monkey playing football', 'An elephant playing hockey', 'A giraffe playing baseball', 'A rhino playing cricket', 'A bear playing volleyball', 'A lion playing rugby', 'A fox playing kickball', 'A hippo playing badminton', 'A koala playing basketball', 'A tiger playing soccer', 'A parrot playing lacrosse', 'A hippo playing tennis', 'A koala playing dodgeball', 'A bear playing ultimate frisbee', 'A lion playing squash', 'A cheetah playing golf', 'A gorilla playing softball', 'A fox playing curling', 'A kangaroo playing handball', 'A crocodile playing water polo', 'A monkey playing table tennis', 'An elephant playing billiards', 'A giraffe playing darts', 'A rhino playing disc golf', 'A bear playing bowling', 'A lion playing badminton', 'A fox playing cricket', 'A hippo playing croquet', 'A koala playing hockey', 'A tiger playing rugby', 'A parrot playing basketball', 'A hippo playing soccer', 'A koala playing ultimate frisbee', 'A bear pla

In [68]:
# create a summary
prompt = "Create a summary for a coloring book about "+topic
summary = generate_summary(prompt)
summary

'This fun and creative coloring book follows anthropomorphic zoo animals as they enjoy playing sports together. With illustrations of animals like tigers, monkeys, and elephants playing soccer, basketball, and more, this book is sure to be a hit with kids of all ages. Enjoy coloring the animals as they cheer each other on and have a great time!'

In [ ]:
# create fun facts and images for each page
fun_facts = []
images = []
count = 0
# iterate for each desired page
for page in pages:
    prompt = "photo, anthropormophic "+page
    # improve the image prompt with a model
    new_prompt = generate(prompt)
    # iterate for the desired number of images to choose from for each page
    for image_num in range(0, images_to_generate):
        # create an image based on the improved prompt
        image = pipe(new_prompt).images[0]
        # enhance/upscale the generated image
        if enhance_images:
            enhanced_image = enhance_image(image)
            # create a coloring book image based on the generated image
            coloring_book_image = generate_coloring_page(enhanced_image)
        else:
            coloring_book_image = generate_coloring_page(image)
        # save the image with the format image_pageNumber_imageNumber as jpg
        coloring_book_image.save("image_"+str(count)+"_"+str(image_num)+".jpg")
        image.save("photo_"+str(count)+"_"+str(image_num)+".jpg")
    # add the fun facts for each page in a list
    fun_facts.append(remove_after_create(generate_fun_facts(page)))
    count = count + 1

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# write to a word document
# create a new docx document object
document = Document()

# set the document size and margins
section = document.sections[0]
section.page_width = Inches(6.0)
section.page_height = Inches(9.0)
section.left_margin = Inches(0.25)
section.right_margin = Inches(0.25)
section.top_margin = Inches(0.5)
section.bottom_margin = Inches(0.5)

# Add a title page
document.add_heading(title, 0)
p = document.add_paragraph('A coloring book about '+topic)
p = document.add_paragraph(summary)

count = 0
for page in pages:
    document.add_heading(pages[count], level=1)
    document.add_page_break()
    document.add_picture("image_"+str(count)+"_0.jpg", width=Inches(5.5))
    document.add_page_break()
    document.add_paragraph('Fun facts about '+pages[count], style='Intense Quote')
    document.add_paragraph(fun_facts[count])
    count += 1

document.save('demo.docx')